In [1]:
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras import layers, Sequential
import numpy as np

2025-06-12 15:07:32.575365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749740852.811127      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749740852.879878      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import tensorflow as tf



In [3]:
dataset_dir = "/kaggle/input/stanford-dogs-dataset/images/Images"
batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.3
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

num_val_test_batches = tf.data.experimental.cardinality(val_test_ds).numpy()
num_test_batches = int(num_val_test_batches * 0.5)
val_ds = val_test_ds.skip(num_test_batches)
test_ds = val_test_ds.take(num_test_batches)

Found 20580 files belonging to 120 classes.
Using 14406 files for training.


I0000 00:00:1749740905.969991      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749740905.970876      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 20580 files belonging to 120 classes.
Using 6174 files for validation.


In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = EfficientNetV2L(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
    classes=120
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(120, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[early_stopping]
)

473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5


I0000 00:00:1749741008.294425     107 service.cc:148] XLA service 0x7b1df8004400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749741008.295412     107 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749741008.295432     107 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749741017.540828     107 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749741070.736217     107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


451/451 ━━━━━━━━━━━━━━━━━━━━ 398s 572ms/step - accuracy: 0.7074 - loss: 1.7158 - val_accuracy: 0.9039 - val_loss: 0.3307
Epoch 2/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 163s 361ms/step - accuracy: 0.8946 - loss: 0.3635 - val_accuracy: 0.9068 - val_loss: 0.3291
Epoch 3/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 167s 369ms/step - accuracy: 0.8955 - loss: 0.3410 - val_accuracy: 0.9065 - val_loss: 0.3327
Epoch 4/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 166s 369ms/step - accuracy: 0.9050 - loss: 0.2930 - val_accuracy: 0.9101 - val_loss: 0.3389
Epoch 5/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 167s 369ms/step - accuracy: 0.9136 - loss: 0.2742 - val_accuracy: 0.9059 - val_loss: 0.3585


In [5]:
model.evaluate(test_ds)

96/96 ━━━━━━━━━━━━━━━━━━━━ 26s 272ms/step - accuracy: 0.8968 - loss: 0.3652


[0.32712051272392273, 0.9052734375]

In [6]:
num_classes = len(train_ds.class_names)
class_names = train_ds.class_names

In [11]:
model.save('model-efficientnetv2l.keras')

In [8]:
maltese_url = "https://teketrek.net/wp-content/uploads/2024/05/%D9%83%D9%84%D8%A8-%D8%A7%D9%84%D9%85%D8%A7%D9%84%D8%B7%D9%8A.webp"
maltese_path = tf.keras.utils.get_file('Maltese', origin=maltese_url)

97714/97714 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [9]:
img = tf.keras.utils.load_img(
    maltese_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [10]:
predictions = model.predict(img_array)
score = predictions[0]

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
This image most likely belongs to n02085936-Maltese_dog with a 94.42 percent confidence.
